# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission, if necessary. Sections that begin with **'Implementation'** in the header indicate where you should begin your implementation for your project. Note that some sections of implementation are optional, and will be marked with **'Optional'** in the header.

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---

## Step 1: Dataset Exploration

Visualize the German Traffic Signs Dataset. This is open ended, some suggestions include: plotting traffic signs images, plotting the count of each sign, etc. Be creative!


The pickled data is a dictionary with 4 key/value pairs:

- features -> the images pixel values, (width, height, channels)
- labels -> the label of the traffic sign
- sizes -> the original width and height of the image, (width, height)
- coords -> coordinates of a bounding box around the sign in the image, (x1, y1, x2, y2)

In [27]:
# Load pickled data
import pickle

# TODO: fill this in based on where you saved the training and testing data
training_file = './train.p'
testing_file = './test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [28]:
### To start off let's do a basic data summary.

# TODO: number of training examples
n_train = len(X_train)

# TODO: number of testing examples
n_test = len(X_test)

# TODO: what's the shape of an image?
image_shape = X_train.shape[1:3]

# TODO: how many classes are in the dataset
n_classes = max(y_train)+1

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)
print("Starting datatype =", type(X_train))

Number of training examples = 39209
Number of testing examples = 12630
Image data shape = (32, 32)
Number of classes = 43
Starting datatype = <class 'numpy.ndarray'>


In [29]:
### Data exploration visualization goes here.
### Feel free to use as many code cells as needed.
sign_dict = {}
for label_sets in [y_train, y_test]:
    for y in label_sets: 
        if y not in sign_dict:
            sign_dict[y] = 1
        else:
            sign_dict[y] +=1

import operator
print("Most common class: %s" % max(sign_dict.items(), key=operator.itemgetter(1))[0])
print("Least common class: %s" % min(sign_dict.items(), key=operator.itemgetter(1))[0])
for key, value in sign_dict.items(): 
    print("%s instances of class %s" % (value, key))

Most common class: 2
Least common class: 0
270 instances of class 0
2940 instances of class 1
3000 instances of class 2
1860 instances of class 3
2640 instances of class 4
2490 instances of class 5
570 instances of class 6
1890 instances of class 7
1860 instances of class 8
1950 instances of class 9
2670 instances of class 10
1740 instances of class 11
2790 instances of class 12
2880 instances of class 13
1050 instances of class 14
840 instances of class 15
570 instances of class 16
1470 instances of class 17
1590 instances of class 18
270 instances of class 19
450 instances of class 20
420 instances of class 21
510 instances of class 22
660 instances of class 23
360 instances of class 24
1980 instances of class 25
780 instances of class 26
300 instances of class 27
690 instances of class 28
360 instances of class 29
600 instances of class 30
1050 instances of class 31
300 instances of class 32
899 instances of class 33
540 instances of class 34
1590 instances of class 35
510 instances

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

There are various aspects to consider when thinking about this problem:

- Your model can be derived from a deep feedforward net or a deep convolutional network.
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [30]:
### Preprocess the data
import tensorflow as tf
import numpy as np

# Min-Max scaling for greyscale image data
def normalize(image_data):
    """
    Normalize the image data 
    """
    return  (image_data - 255)/255


# NOTE: Same steps are done later for other sets
X_test_tf  = tf.squeeze(tf.image.rgb_to_grayscale(X_test))
X_test_tf = tf.to_float(X_test_tf)
sess = tf.Session()
with sess.as_default() as s: X_test_np = X_test_tf.eval()
test_features_tf  = tf.map_fn(normalize, X_test_tf)
print(type(X_test_np))
test_labels_tf = tf.convert_to_tensor(y_test)

<class 'numpy.ndarray'>


### Question 1 

_Describe the techniques used to preprocess the data._

**Answer: I did some basic normalization on the data. I converted the images to grayscale. I also zero-centered the data and gave it equal variance (per the Deep Learning slides). Additionally, I converted the labels to one-hot encoding.**

In [31]:
### Generate data additional (if you want to!)
### TODO: If problems, DO THIS

### split the data into training/validation/testing sets here.
# Get randomized datasets for training and validation
from sklearn.cross_validation import train_test_split
train_features_np, valid_features_np, train_labels_np, valid_labels_np = train_test_split(
    X_train,
    y_train,
    test_size=0.10,
    random_state=832289)

train_features_tf = tf.squeeze(tf.image.rgb_to_grayscale(train_features_np))
valid_features_tf = tf.squeeze(tf.image.rgb_to_grayscale(valid_features_np))
train_features_tf = tf.to_float(train_features_tf)
valid_features_tf = tf.to_float(valid_features_tf)
train_features_tf = tf.map_fn(normalize, train_features_tf)
valid_features_tf = tf.map_fn(normalize, valid_features_tf)

temp_sess = tf.Session()
with temp_sess.as_default() as s: valid_features_np = valid_features_tf.eval()

    
# Change to float32, so it can be multiplied against the features in TensorFlow, which are float32
train_labels = train_labels_np.astype(np.float32)
valid_labels = valid_labels_np.astype(np.float32)

### Question 2

_Describe how you set up the training, validation and testing data for your model. If you generated additional data, why?_

**Answer: Test data was already seperate. Shuffled the training set and divided it into training and validation (with validation being 10% of the size of the original training set) Additionally, I did the same preprocessing I had done on the test set after making the training/validation split.**

In [32]:
import math
### Define architecture here.
num_channels = 1 # grayscale
image_size = 32
dropout = True

#def dense_to_one_hot(labels_dense, num_classes=43):
#    """Convert class labels from scalars to one-hot vectors."""
#    num_labels = labels_dense.shape[0]
#    index_offset = np.arange(num_labels) * num_classes
#    labels_one_hot = np.zeros((num_labels, num_classes))
#    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
#    return labels_one_hot

def dense_to_one_hot(labels_dense, num_classes=43):
        num_labels = labels_dense.shape[0]
        index_offset = np.arange(num_labels) * num_classes
        labels_one_hot = np.zeros((num_labels, num_classes))
        labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
        return labels_one_hot

def reformat(dataset, labels):
    dataset = dataset.reshape(
      (-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = dense_to_one_hot(labels) #, n_classes).astype(np.float32) 
    #(np.arange(n_classes) == labels[:,None]).astype(np.float32)
    return dataset, labels

def conv2d(x, W):
    return tf.nn.conv2d(x, W, 
                        strides=[1, 2, 2, 1], 
                        padding='SAME')

def max_pool(x, k):
    return tf.nn.max_pool(x, 
                          ksize=[1, k, k, 1], 
                          strides=[1, k, k, 1], 
                          padding='SAME')


# Parameters
display_step = 1


#n_input = 1024  # traffic sign image data input (img shape: 32*32)
n_classes = 43  # total classes (43 signs)
convolutional = True
dropout = False
input_size = 32*32*1

train_features, train_labels = reformat(train_features_np, train_labels_np)
valid_features, valid_labels = reformat(valid_features_np, valid_labels_np)
test_features,  test_labels  = reformat(X_test, y_test)
print(valid_features.shape)
test_shape = X_test_np.shape
graph = tf.Graph()

if convolutional:
    batch_size = 100
    patch_size = 8
    depth = 32
    num_hidden = 64

    
    with graph.as_default():
        
        # Input data.
        graph_features = tf.placeholder(tf.float32, 
                                        shape=(None, #batch_size, 
                                               image_size, image_size, num_channels))
        graph_labels = tf.placeholder(tf.float32, shape=(None, #batch_size, 
                                                         n_classes))
        keep_prob = tf.placeholder(tf.float32) # for dropout
        

        # Model.
        W_convl1 = tf.Variable(# 5x5 pixel patch, depth 3, 4 feature maps
                     tf.truncated_normal(shape=[patch_size,patch_size,num_channels,depth], 
                                      stddev=1e-4))
        b_convl1 = tf.constant(0.1, shape=[depth]) # 4 feature maps
        h_convl1 = tf.nn.relu(conv2d(graph_features, W_convl1) + b_convl1)
            
            
        h_pool1 = max_pool(h_convl1, 2)
        if dropout: 
            h_pool1 = tf.nn.dropout(h_pool1, keep_prob)
            
        W_convl2 = tf.Variable(# 5x5 pixel patch, depth 4, 6 feature maps
                   tf.truncated_normal(shape=[patch_size,patch_size,32,depth], #6], 
                                       stddev=1e-4))
        b_convl2 = tf.constant(0.1, shape=[depth]) 
        h_convl2 = tf.nn.relu(conv2d(h_pool1, W_convl2) + b_convl2)
            
        h_pool2 = max_pool(h_convl2, 2)
    
        if dropout:
            tf.nn.dropout(h_pool2, keep_prob)
                
        h_w = tf.Variable(
              tf.truncated_normal(
                  [4*W_convl2.get_shape().as_list()[-1], 1024],
                  stddev=1.0/math.sqrt(float(input_size))), name='hidden_weights')

        h_pool2 = tf.reshape(h_pool2, [-1, h_w.get_shape().as_list()[0]])
        h_b = tf.Variable(tf.zeros([1024]),
                              name='hidden_biases')
        hidden = tf.nn.relu(tf.nn.bias_add(tf.matmul(h_pool2, 
                                                         h_w), h_b))
        if dropout:
            hidden = tf.nn.dropout(hidden, keep_prob)
                
        p_w = tf.Variable(
              tf.truncated_normal( 
                  [1024, n_classes],
                  stddev=1.0),            
              name='output_weights')
        p_b = tf.Variable(tf.zeros([n_classes]),
                              name='output_biases')
        logits = tf.add(tf.matmul(hidden, p_w), p_b)
            
        
        def model_old(data):
            conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], 
                                padding='SAME')
            hidden = tf.nn.relu(conv + layer1_biases)
            conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], 
                                padding='SAME')
            hidden = tf.nn.relu(conv + layer2_biases)
            shape = hidden.get_shape().as_list()
            reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
            
            hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
            
            return tf.matmul(hidden, layer4_weights) + layer4_biases


(3921, 32, 32, 1)


### Question 3

_What does your final architecture look like? (Type of model, layers, sizes, connectivity, etc.)  For reference on how to build a deep neural network using TensorFlow, see [Deep Neural Network in TensorFlow
](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/b516a270-8600-4f93-a0a3-20dfeabe5da6/concepts/83a3a2a2-a9bd-4b7b-95b0-eb924ab14432) from the classroom._


**Answer:**

In [33]:
learning_rate = 0.01
reg_strength = 0.1

### Train model here.
with graph.as_default():
    loss = (tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, graph_labels)))
    
    regularizers  = tf.nn.l2_loss(W_convl1) + tf.nn.l2_loss(b_convl1)
    regularizers += tf.nn.l2_loss(W_convl2) + tf.nn.l2_loss(b_convl2)
    regularizers += tf.nn.l2_loss(h_w) + tf.nn.l2_loss(h_b)
    regularizers += tf.nn.l2_loss(p_w) + tf.nn.l2_loss(p_b)
    loss += reg_strength*regularizers

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    prediction = tf.nn.softmax(logits)
    
    correct_pred = tf.equal(tf.argmax(logits, 1), 
                            tf.argmax(graph_labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


num_steps = 40000
do_restore = False
prob_keep = 0.90 

with tf.Session(graph=graph) as session:
    init = tf.initialize_all_variables()
    session.run(init)
    
    saver = tf.train.Saver()
    #if do_restore:
    #    saver.restore(sess, "/tmp/model.ckpt")
    
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_features[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {
                     graph_features : batch_data,
                     graph_labels   : batch_labels,
                     keep_prob      : prob_keep,
                    } 
        _, l = session.run([optimizer, loss], feed_dict=feed_dict)
        
        if (step % 100 == 0):
            print('Loss %s (step=%s)' % (l, step))
            
            t_acc = session.run(accuracy, feed_dict={
                    graph_features: batch_data,
                    graph_labels: batch_labels})     
            print("Training Accuracy: %s" % t_acc)
                           
            v_acc = session.run(accuracy, feed_dict={graph_features: valid_features,
                                                       graph_labels: valid_labels})
            print("Validation Accuracy: %s" % v_acc)

                           
    test_acc = sess.run(accuracy, feed_dict={graph_features:X_test_np, 
                                             graph_labels: valid_labels})
    print("Test Accuracy: %s" % test_acc)
                           
                           
    #save_path = saver.save(sess, "/tmp/model.ckpt")
            

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Loss 1699.24 (step=0)
Training Accuracy: 0.09
Validation Accuracy: 0.0566182
Loss nan (step=100)
Training Accuracy: 0.0
Validation Accuracy: 0.00535578
Loss nan (step=200)
Training Accuracy: 0.01
Validation Accuracy: 0.00535578
Loss nan (step=300)
Training Accuracy: 0.0
Validation Accuracy: 0.00535578
Loss nan (step=400)
Training Accuracy: 0.0
Validation Accuracy: 0.00535578
Loss nan (step=500)
Training Accuracy: 0.01
Validation Accuracy: 0.00535578
Loss nan (step=600)
Training Accuracy: 0.01
Validation Accuracy: 0.00535578
Loss nan (step=700)
Training Accuracy: 0.0
Validation Accuracy: 0.00535578
Loss nan (step=800)
Training Accuracy: 0.01
Validation Accuracy: 0.00535578
Loss nan (step=900)
Training Accuracy: 0.0
Validation Accuracy: 0.00535578
Loss nan (step=1000)
Training Accuracy: 0.01
Validation Accuracy: 0.00535578
Loss nan (step=1100)
Training Accuracy: 0.02
Validation Accuracy: 0.00535578
Loss

KeyboardInterrupt: 

### Question 4

_How did you train your model? (Type of optimizer, batch size, epochs, hyperparameters, etc.)_


**Answer:**

### Question 5


_What approach did you take in coming up with a solution to this problem?_

**Answer:**

---

## Step 3: Test a Model on New Images

Take several pictures of traffic signs that you find on the web or around you (at least five), and run them through your classifier on your computer to produce example results. The classifier might not recognize some local signs but it could prove interesting nonetheless.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

### Question 6

_Choose five candidate images of traffic signs and provide them in the report. Are there any particular qualities of the image(s) that might make classification difficult? It would be helpful to plot the images in the notebook._



**Answer:**

In [ ]:
### Run the predictions here.
### Feel free to use as many code cells as needed.

### Question 7

_Is your model able to perform equally well on captured pictures or a live camera stream when compared to testing on the dataset?_


**Answer:**

In [ ]:
### Visualize the softmax probabilities here.
### Feel free to use as many code cells as needed.

### Question 8

*Use the model's softmax probabilities to visualize the **certainty** of its predictions, [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.11/api_docs/python/nn.html#top_k) could prove helpful here. Which predictions is the model certain of? Uncertain? If the model was incorrect in its initial prediction, does the correct prediction appear in the top k? (k should be 5 at most)*


**Answer:**

### Question 9
_If necessary, provide documentation for how an interface was built for your model to load and classify newly-acquired images._


**Answer:**

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.